In [ ]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can uncomment the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)

# !sudo ldconfig
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4f606ac964b5e760c9c4fc17733c286c75e2a6b50c254d134bb60ed827572ff1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 816.3 kB/s eta 0:00:00
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [118]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# PageRank

In [119]:
Web_Google = spark.sparkContext.textFile("/content/web-Google.txt")

In [120]:
edges = Web_Google.filter(lambda line: not line.startswith('#'))
edges = edges.map(lambda line: tuple(map(int, line.split())))
edges.take(3)

[(0, 11342), (0, 824020), (0, 867923)]

In [121]:
outGoing_links = edges.distinct().groupByKey()

In [128]:
nodes = edges.flatMap(lambda edge: edge).distinct()

In [129]:
print(f"Number of nodes: {nodes.count()}")

Number of nodes: 875713


In [130]:
nodes_number = nodes.count()
initial_rank = 1 / nodes_number

In [145]:
# initial to each node
rank = nodes.map(lambda node: (node, initial_rank))

In [146]:
#damping factor
alpha = 0.85
for i in range(10):

    rank = outGoing_links.join(rank).flatMap( lambda x: [(i, x[1][1] / len(x[1][0])) for i in x[1][0]])\
    .reduceByKey(lambda a,b: a+b)\
    .mapValues(lambda rank_sum: alpha * rank_sum + ((1 - alpha) / nodes_number))

In [101]:
final_ranks = rank.collect()

In [142]:
# Save the final ranks to a file
with open("final_ranks.txt", "w") as file:
    for node_id, page_rank in final_ranks:
        file.write(f"Node ID: {node_id}, PageRank: {page_rank}\n")

In [147]:
sorted_rank = rank.sortBy(lambda x: x[1], ascending=False)

# Take top 10 elements
top_10_pages = sorted_rank.take(10)

# Print the top 10 pages
for page in top_10_pages:
    print("Page:", page[0], "PageRank:", page[1])

Page: 41909 PageRank: 0.0005089770118389987
Page: 597621 PageRank: 0.0004643397628564387
Page: 504140 PageRank: 0.00045573071171432913
Page: 384666 PageRank: 0.000448578294179169
Page: 537039 PageRank: 0.0004383960561316232
Page: 486980 PageRank: 0.00043640513591067167
Page: 751384 PageRank: 0.00041316451455076917
Page: 32163 PageRank: 0.0004127175181650983
Page: 163075 PageRank: 0.0004087180742310319
Page: 605856 PageRank: 0.00040720779003414307
